# Overview:

This NB combines all parts of cleaned data to form the data to be used for regression

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import datetime as dt
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.sql import text

## First, we get data on 

* `all_student_features`: all non-class features by student-day (without the frac_class_positive covariate), including whether the student positive on that day (or whether he/she is a previous positive)
* `all_student_classes`: students and the classes they take
* `positive_students`: student positive cases with hd_notify_date between 8/26 and 12/7
* `class_schedule`: class sessions, meeting times, enrollment counts

In [2]:
all_students_classes = pd.read_csv('do_not_touch/class_registration.csv')

In [3]:
print(all_students_classes.shape)
all_students_classes.head()

(143873, 12)


,employee_id_hash,is_greek,is_athlete,academic_career,academic_plan1,subject,catalog_nbr,class_number,class_enroll_tot,class_section,student_infected,hd_notify_date
0,0x0001CEED0A3584312155FD3B695D2EB6,0,0,UG,ESAG-BS,PLSCS,2200.0,2885.0,23.0,401,0,NaN
1,0x0001CEED0A3584312155FD3B695D2EB6,0,0,UG,ESAG-BS,BIOEE,4620.0,17542.0,36.0,1,0,NaN
2,0x0001CEED0A3584312155FD3B695D2EB6,0,0,UG,ESAG-BS,MATH,2210.0,4310.0,58.0,1,0,NaN
3,0x0001CEED0A3584312155FD3B695D2EB6,0,0,UG,ESAG-BS,PLSCS,2200.0,2884.0,44.0,1,0,NaN
4,0x0001CEED0A3584312155FD3B695D2EB6,0,0,UG,ESAG-BS,QUECH,1210.0,18873.0,11.0,101,0,NaN


In [12]:
all_students_features = pd.read_csv('do_not_touch/all_students_features_trunc_corrected.csv', index_col = 0) 
print(all_students_features.shape)
all_students_features.head()

C:\Users\yz685\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(2891563, 13)


,employee_id_hash,is_greek,is_athlete,academic_career,academic_plan1,hd_notify_date,day_idx,positives_identified_on_this_day,infected_on_this_day,previous_infection,week_idx,biweek_idx,class_prevalence_on_this_day
0,0x0001CEED0A3584312155FD3B695D2EB6,0,0,UG,ESAG-BS,NaN,2021-08-26,23,0,0,0,0,NaN
1,0x0001CEED0A3584312155FD3B695D2EB6,0,0,UG,ESAG-BS,NaN,2021-08-27,43,0,0,0,0,NaN
2,0x0001CEED0A3584312155FD3B695D2EB6,0,0,UG,ESAG-BS,NaN,2021-08-28,57,0,0,0,0,NaN
3,0x0001CEED0A3584312155FD3B695D2EB6,0,0,UG,ESAG-BS,NaN,2021-08-29,55,0,0,0,0,NaN
4,0x0001CEED0A3584312155FD3B695D2EB6,0,0,UG,ESAG-BS,NaN,2021-08-30,39,0,0,0,0,NaN


In [13]:
# old, this is to be updated
# G/A status not trustworthy for now
all_students_features['academic_career'].value_counts()

UG      1295444
GR       899232
UG_G     255188
GM       180414
UG_A     112471
LA        96506
VM        52204
EE          104
Name: academic_career, dtype: int64

In [7]:
positive_students = pd.read_csv('do_not_touch/positive_students.csv', index_col = 0) 
print(positive_students.shape)
positive_students.head()

(714, 6)


,employee_id_hash,hd_notify_date,is_greek,is_athlete,ncaa_sport_1,academic_career
0,0xE00F3E92C639BDD0BBD99E9A13CD03F4,2021-09-09 00:00:00.0000000,1,0,NaN,UG
1,0xF195D94DD0780C71FB76CC288A59CED1,2021-09-10 00:00:00.0000000,1,0,NaN,UG
2,0x269C4236CB3ED23CF2CDDAEFE04C8295,2021-08-27 00:00:00.0000000,1,0,NaN,UG
3,0x2E5315A557040640A6FA044D6F40F5FB,2021-08-28 00:00:00.0000000,1,0,NaN,UG
4,0xEAD9251A405B52B7143EE3D12112FCF2,2021-09-02 00:00:00.0000000,1,0,NaN,UG


In [8]:
class_schedule = pd.read_csv('do_not_touch/class_schedule.csv', index_col = 0)
class_schedule.shape

(5631, 14)

In [11]:
agg_daily_class_prevalence = pd.read_csv('do_not_touch/agg_daily_class_prevalence.csv', index_col = 0)
agg_daily_class_prevalence.head()

C:\Users\yz685\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [14]:
campus_positivity = pd.read_csv('do_not_touch/campus_positivity.csv', index_col = 0)
campus_positivity.head()

C:\Users\yz685\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,campus_positivity_backward,campus_positivity_forward_backward
0,296.0,296.0
1,315.0,338.0
2,298.0,364.0
3,270.0,393.0
4,219.0,397.0


In [18]:
all_students_features.drop(columns=['class_prevalence_on_this_day'], inplace=True)

In [20]:
all_students_features['class_positivity'] = agg_daily_class_prevalence
all_students_features['campus_positivity_backward'] = campus_positivity['campus_positivity_backward']
all_students_features['campus_positivity_forward_backward'] = campus_positivity['campus_positivity_forward_backward']


In [22]:
all_students_features.head()

,employee_id_hash,is_greek,is_athlete,academic_career,academic_plan1,hd_notify_date,day_idx,positives_identified_on_this_day,infected_on_this_day,previous_infection,week_idx,biweek_idx,class_positivity,campus_positivity_backward,campus_positivity_forward_backward
0,0x0001CEED0A3584312155FD3B695D2EB6,0,0,UG,ESAG-BS,NaN,2021-08-26,23,0,0,0,0,0.0,296.0,296.0
1,0x0001CEED0A3584312155FD3B695D2EB6,0,0,UG,ESAG-BS,NaN,2021-08-27,43,0,0,0,0,0.0,315.0,338.0
2,0x0001CEED0A3584312155FD3B695D2EB6,0,0,UG,ESAG-BS,NaN,2021-08-28,57,0,0,0,0,0.0,298.0,364.0
3,0x0001CEED0A3584312155FD3B695D2EB6,0,0,UG,ESAG-BS,NaN,2021-08-29,55,0,0,0,0,0.0,270.0,393.0
4,0x0001CEED0A3584312155FD3B695D2EB6,0,0,UG,ESAG-BS,NaN,2021-08-30,39,0,0,0,0,0.0,219.0,397.0


In [23]:
all_students_features.to_csv('do_not_touch/all_covariates_finalized.csv')